In [ ]:
## 1. 고속도로 출입시설 위치정보 

import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
from pprint import pprint
 
api_key = '1044310891'
default_url = 'http://data.ex.co.kr/openapi/locationinfo/locationinfoIc?' 
query = 'key=' + api_key + '&' + 'type=json'
url = default_url + query

request = Request(url)
response = urlopen(request)

rescode = response.getcode()

icName_list = []
x_list = []
y_list = []
routeName_list = []

if rescode == 200:
    response_body = response.read().decode('utf-8')
    
    dicts = json.loads(response_body)
    for dict in dicts["list"]:
        icName_list.append(dict['icName'])
        x_list.append(dict['xValue'])
        y_list.append(dict['yValue'])
        routeName_list.append(dict['routeName'])
else:
    print('Response error code : ',rescode)

In [ ]:
data = {
    'icName' : icName_list,
    'x_coor' : x_list,
    'y_coor' : y_list,
    'routeName' : routeName_list
}

df = pd.DataFrame(data)
df.head()

In [ ]:
df.to_csv('고속도로_출입시설_위치정보.csv')

In [ ]:
import pandas as pd

#2. 고속도로 시각화

    #2 - 1. 데이터 전처리
df = pd.read_csv('도로종류별-교통량_병권님수정본.csv')
df.head()

In [ ]:
df.columns = ['노선번호','구간','이정','연장','2017','2018','2019','2020','계','1~2종','3~12종']
df.head()

In [ ]:
import re 

df['구간'] = df['구간'].apply(lambda x : re.sub(' ','',x))
df.head()

In [ ]:
df.info()

In [ ]:
df['2017'] = df['2017'].apply(lambda x: re.sub(' ','',x))
df['2018'] = df['2018'].apply(lambda x: re.sub(' ','',x))
df['2019'] = df['2019'].apply(lambda x: re.sub(' ','',x))
df['2020'] = df['2020'].apply(lambda x: re.sub(' ','',x))

In [ ]:
import numpy as np

df.replace('-',np.nan,inplace=True)
df.head()

In [ ]:
df['2017'].fillna(0,inplace=True)
df['2018'].fillna(0,inplace=True)
df['2019'].fillna(0,inplace=True)
df['2020'].fillna(0,inplace=True)

In [ ]:
df.head()

In [ ]:
df['2017'] = df['2017'].apply(lambda x: re.sub(',','',str(x)))
df['2018'] = df['2018'].apply(lambda x: re.sub(',','',str(x)))
df['2019'] = df['2019'].apply(lambda x: re.sub(',','',str(x)))
df['2020'] = df['2020'].apply(lambda x: re.sub(',','',str(x)))
df['1~2종'] = df['1~2종'].apply(lambda x: re.sub(',','',str(x)))
df['3~12종'] = df['3~12종'].apply(lambda x: re.sub(',','',str(x)))

df['2017'] = df['2017'].astype(int)
df['2018'] = df['2018'].astype(int)
df['2019'] = df['2019'].astype(int)
df['2020'] = df['2020'].astype(int)
df['1~2종'] = df['1~2종'].astype(int)
df['3~12종'] = df['3~12종'].astype(int)

df.head()

In [ ]:
df.info()

In [ ]:
df['17~20년도_합계'] = df['2017'] + df['2018'] + df['2019'] + df['2020'] 

In [ ]:
df

In [ ]:
mean_2017 = df['2017'].mean()
mean_2018 = df['2018'].mean()
mean_2019 = df['2019'].mean()
mean_2020 = df['2020'].mean()

df['2017'] = df['2017'].apply(lambda x: mean_2017 if x == 0 else x)
df['2018'] = df['2018'].apply(lambda x: mean_2017 if x == 0 else x)
df['2019'] = df['2019'].apply(lambda x: mean_2017 if x == 0 else x)
df['2020'] = df['2020'].apply(lambda x: mean_2017 if x == 0 else x)

In [ ]:
df['17~20년도_합계'] = df['2017'] + df['2018'] + df['2019'] + df['2020']
df.head()

In [ ]:
#2.2 구간 좌표 추출

startpoint = []
endpoint = []

for sector in df['구간']:
    
    start = sector.split('~')[0]
    end = sector.split('~')[1]

    startpoint.append(start)
    endpoint.append(end)
    
df['startpoint'] = startpoint
df['endpoint'] = endpoint

In [ ]:
import requests

def to_latlng2(place):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(place)
    headers = { "Authorization": "KakaoAK " + "583cc3a933b33ebd2e55e3dc318ea19b" }
    places = requests.get(url, headers = headers).json()
    target_x = 0
    target_y = 0
    
    for place in places['documents']:
        if place['category_name'].find('교통,수송') == -1:
            pass
        else:
            target_x = place['x']
            target_y = place['y']
            break
    
    xVal = float(target_x)
    yVal = float(target_y)
    
    return (yVal, xVal)

In [ ]:
to_latlng2('통도사IC')

In [ ]:
startpoint_list = []
endpoint_list = []

for start, end in zip(df['startpoint'].tolist(), df['endpoint'].tolist()):
    startpoint_list.append(to_latlng2(start))
    endpoint_list.append(to_latlng2(end))
    
    print('[',start,'~',end,'] 완료')

print(len(startpoint_list))
print(len(endpoint_list))

In [ ]:
df['startpoint_coor'] = startpoint_list
df['endpoint_coor'] = endpoint_list

df.head()

In [ ]:
df.to_csv('도로종류별 교통량_좌표추가.csv')

In [ ]:
from pprint import pprint
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format('서울주JC')
headers = { "Authorization": "KakaoAK " + "583cc3a933b33ebd2e55e3dc318ea19b" }
places = requests.get(url, headers = headers).json()

doc = places['documents']
pprint(doc)